# Spatial sampling density variance

<!-- SUMMARY: Spatial sampling density variance: its calculation and usage in mining industry -->

<!-- CATEGORY: Methodology -->

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc

gdoc.setNoScroll()

# Introduction

This document is meant to give some introductory information on various tools derived from the geostatistical etimation variance, which have been developed to measusre the sampling density, either for a regular sampling design or an irregular one.
It follows the paper entitled *From the Spatial Sampling of a Deposit to Mineral Resources Classification*, from Jacques Rivoirard, Didier Renard, Felipe Celhay, David Benado, Celeste Queiroz, Leandro Jose Oliveira, and Diniz Ribeiro, presented in Geostatistics Valencia 2016.

## The concepts

In this paper, $Z(x)$ denotes the regionalized (additive) variable. Considering a domain V, the estimation  of $Z(V)$  by inner samples of $V$ is denoted $Z(V)^*$ and the corresponding estimation variance $\sigma_E^2 (V)$ which is calculated from the variogram as follows:
$$\sigma_E^2 (V)={2 \over N} \sum_i \bar \gamma(x_i,V) - {1 \over N^2} \sum_{ij} \gamma(x_i,x_j) - \bar \gamma(V,V)$$

The spatial sampling density variance (**ssdv**) corresponds is $$\chi(V) = {\sigma_E^2 (V) \times |V|}$$. 


The specific volume measures the density of a spatial sampling of the variable, relatively to its mean: the smaller the specific volume, the more precise the sampling pattern:
$$V_0 = { {\sigma_E^2 (V) \times |V|} \over M^2 }$$

The coefficient of variation for a given Volume is:
$$CoV = \sqrt{ V_0 \over V}$$

The specific area corresponding to a given CoV (area over which the mean variable has a given coefficient of variation) is:
$$V = { V_0 \over CoV^2 }$$

## Application to regular sampling

The previous concepts are be illustrated using the following example.

Consider a large deposit in two dimensions, sampled with regular grid. the variable of interest is the thickness (with a mean of 10m), a sample point variance of 10, and a variogram equal to: $$1 nug(h) + 9sph(h/200m)$$


In [ ]:
mean_10 = 10

model1 = gl.Model.createFromParam(gl.ECov.NUGGET,sill=1)
model1.addCovFromParam(gl.ECov.SPHERICAL,range=200,sill=9)
model1

Most of the subsequent calculations need the determination of quantities which involve extension variance, the latter being based on calulation of average variogram over a block. This operation is performed by discretization which needs to be specified explicitely:

In [ ]:
ndisc = [30,30]

The extension variance of a $100m \times 100m$ by its center is:

In [ ]:
target = gl.Db.createFromOnePoint([0,0])
block_100 = [100, 100]
Sigma_E = model1.extensionVariance(target, block_100, ndisc)
print("Extension Variance = %1.3f" %Sigma_E)

Therefore, for a $100m \times 100m$ grid, the spatial sampling density variance is:

In [ ]:
ssdv = model1.samplingDensityVariance(target, block_100, ndisc)
print("Spatial Sampling Density Variance = %1.2f m².m²" %ssdv)

The specific area (i.e. the specific volume in two dimensions) is:

In [ ]:
V_100 = model1.specificVolume(target, mean_10, block_100, ndisc)
print("Specific Area (case #1) = %1.2f m²" %V_100)

If the grid is now a $50m \times 50m$, the specific volume becomes:

In [ ]:
block_50 = [50, 50]
V_50 = model1.specificVolume(target, mean_10, block_50, ndisc)
print("Specific Area (small block) = %1.2f m²" %V_50)

Another issue is whether the initial $100m \times 100m$ grid is better, or not, than a $70m \times 70m$ sampling in a similar deposit, where the thickness has a mean of 8 and a variogram equal to $$6 nug(h) + 6sph(h/200m)$$

In [ ]:
mean_8 = 8

model2 = gl.Model.createFromParam(gl.ECov.NUGGET,sill=6)
model2.addCovFromParam(gl.ECov.SPHERICAL,range=200,sill=6)
model2

In [ ]:
block_70 = [70, 70]
V_70 = model2.specificVolume(target, mean_8, block_70, ndisc)
print("Specific Area (case #2) = %1.2f m²" %V_70)

We can finally characterize resources given an assumption on the production volume (or more exactly the
ore + waste volume to be extracted). For instance, a specific volume (3) of 6000 m3 gives a CoV of:

In [ ]:
Volume = 6000
CoV = 100 * model1.coefficientOfVariation(target, Volume, mean_10, block_100, ndisc)
print("Coefficient of Variation = %1.2f" %CoV)

Conversely we can define the specific volume to be mined in order to ensure a given CoV.
For instance we can deduce that the 5% CoV area (area over which the mean thickness
has a coefficient of variation of 5%) must practically reach 200,000 m2, while it is
only108,000 m2 in the first case

In [ ]:
cov_5 = 0.05
Vol1 = model1.specificVolumeFromCoV(target, cov_5, mean_10, block_100, ndisc)
print("Specific area (case #1) = %1.2f m²" %Vol1)
Vol2 = model2.specificVolumeFromCoV(target, cov_5, mean_8, block_70, ndisc)
print("Specific area (case #2) = %1.2f m²" %Vol2)